<div style="font-size:18pt; padding-top:20px; text-align:center; line-height: 1.5;">СЕМИНАР. <b>Оптимизация. Часть 2.</b> Стохастический градиентный спуск и библиотеки <span style="font-weight:bold; color:green">Python</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_hse@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Градиентный спуск для функции с суммой квадратов</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#1a">Функция с суммой квадратов</a></li>
                <li><a href="#1b">Градиентный спуск</a></li>
                <li><a href="#1с">Стохастический градиентный спуск</a></li>
                <li><a href="#1d">Градиентный спуск в задаче линейной регрессии</a></li>
            </ol>
        </li>
        <li><a href="#2">Оптимизация и SciPy</a></li>
        <li><a href="#3">Источники</a></li>
    </ol>
</div>

In [ ]:
import numpy as np
import pandas as pnd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib inline

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Градиентный спуска для функции с суммой квадратов</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name="1a"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Функция с суммой квадратов
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#1b">Далее</a>
            </div>
        </div>
    </div>
</div>

<p>Формирование исходных данных</p>

In [ ]:
np.random.seed(100)
w1 = 0.5 * np.random.randn(1000) + 1
w2 = 0.5 * np.random.randn(1000) + 0.5
y = 0.5 * np.random.randn(1000)

In [ ]:
# Количество элементов в выборке
n = len(y)

# Функция потерь (ошибки)  
f = lambda w1, w2, x1, x2: 1 / n * sum([(w1[i]*x1 + w2[i]*x2 - y[i]) ** 2 for i in range(n)])

In [ ]:
# Частная производная по x1
df_x1 = lambda w1, w2, x1, x2: 2*sum(w1[i]*(w1[i]*x1 + w2[i]*x2 - y[i]) for i in range(n))

# Частная производная по x2
df_x2 = lambda w1, w2, x1, x2: 2*sum(w2[i]*(w1[i]*x1 + w2[i]*x2 - y[i]) for i in range(n))

In [ ]:
coord_x1 = np.arange(-4, 5, 0.1) #Значения x c шагом 0.1
coord_x2 = np.arange(-4, 5, 0.1) #Значения x c шагом 0.1

x1, x2 = np.meshgrid(coord_x1, coord_x2)

In [ ]:
fig = plt.figure(1, figsize=(10, 10))

ax0 = fig.add_subplot(2, 2, 1, projection="3d")
ax0.plot_surface(x1, x2, f(w1, w2, x1, x2), rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax0.set_xlabel("x1")
ax0.set_ylabel("x2")
ax0.set_zlabel("f(x1,x2)")

ax1 = plt.subplot(2,2,2)
cf = ax1.contourf(x1, x2, f(w1, w2, x1, x2), 10, alpha=0.5, cmap=cm.coolwarm)
plt.colorbar(cf)
ax1.set_xlabel("x1")
ax1.set_ylabel("x2")

ax2 = plt.subplot(2,2,3)
ax2.set_xlabel("x1")
ax2.set_ylabel("x2")

X, Y = np.meshgrid(coord_x1[::5], coord_x2[::5])
Fxy = f(w1, w2, X, Y)
dX = coord_x1[5] - coord_x1[0]
dY = coord_x2[5] - coord_x2[0]
dFY,dFX = np.gradient(Fxy, dX, dY)

ax2.quiver(X, Y, dFX, dFY, scale=100)

ax3 = plt.subplot(2,2,4)
ax3.set_xlabel("x1")
ax3.set_ylabel("x2")
ax3.contourf(x1, x2, f(w1, w2, x1, x2), 25, cmap=cm.coolwarm)
ax3.quiver(X, Y, dFX, dFY, scale=100)
plt.colorbar(cf)

plt.tight_layout()

plt.show()

<a name="1b"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Градиентный спуск
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1a">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#1c">Далее</a>
            </div>
        </div>
    </div>
</div>

<p>Алгоритм градиентного спуска</p>

In [ ]:
def gradientDescent2D_Lite(w1, w2, alpha, f, dfx1, dfx2, iterNum, startX1, startX2):
    
    x1_min = startX1
    x1_min_prev = startX1
    
    x2_min = startX2
    x2_min_prev = startX2
    
    #y_prev = f(w1, w2, x1_min, x2_min)
    #y_cur = y_prev
    
    i = 0
    
    while i < iterNum:
        
        x1_min = x1_min_prev - alpha * dfx1(w1, w2, x1_min_prev, x2_min_prev)
        x2_min = x2_min_prev - alpha * dfx2(w1, w2, x1_min_prev, x2_min_prev)
        
        #y_cur = f(w1, w2, x1_min, x2_min)
        
        #if abs(y_cur - y_prev) <= err:
        #    break
        
        #y_prev = y_cur
        
        x1_min_prev = x1_min
        x2_min_prev = x2_min

        i += 1
        
    return (x1_min, x2_min, i)

<p>Поиск решения посредством градиентного спуска</p>

In [ ]:
alpha = 0.0005
iterNum = 200
startX1 = 4
startX2 = -2

In [ ]:
gradientDescent2D_Lite(w1, w2, alpha, f, df_x1, df_x2, iterNum, startX1, startX2)

<p>Исследование влияния значения коэффициента альфа</p>

<p><b><i>Начальная точка 1</i></b></p>

<img src="img/gd-two-var-alpha-right-sum.png">

<p><b><i>Начальная точка 2</i></b></p>

<img src="img/gd-two-var-alpha-left-sum.png">

<p>Время выполнения градиентного спуска</p>

In [ ]:
alpha = 0.0005
iterNum = 200
startX1 = 4
startX2 = -2

In [ ]:
%timeit gradientDescent2D_Lite(w1, w2, alpha, f, df_x1, df_x2, iterNum, startX1, startX2)

<a name = "1c"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            c. Стохастический градиентный спуск
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1b">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#1d">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
# Частные производные от функции потерь
df_x1_i = lambda w1, w2, x1, x2, i: 1/2.0 * w1[i] * (w1[i]*x1 + w2[i]*x2 - y[i])
df_x2_i = lambda w1, w2, x1, x2, i: 1/2.0 * w2[i] * (w1[i]*x1 + w2[i]*x2 - y[i])

<p>Алгоритм стохастического градиентного спуска</p>

In [ ]:
def stochasticGradientDescent2D_Lite(w1, w2, alpha, f, dfx1i, dfx2i, err, startX1, startX2):
    
    n = len(y)
    
    x1_min = startX1
    x1_min_prev = startX1
    
    x2_min = startX2
    x2_min_prev = startX2
    
    #y_prev = f(w1, w2, x1_min, x2_min)
    #y_cur = y_prev
    
    j = 0
    
    for i in range(n):
        
        x1_min = x1_min_prev - alpha * dfx1i(w1, w2, x1_min_prev, x2_min_prev, i)
        x2_min = x2_min_prev - alpha * dfx2i(w1, w2, x1_min_prev, x2_min_prev, i)
        
        #y_cur = f(w1, w2, x1_min, x2_min)
        
        #if abs(y_cur - y_prev) <= err:
        #    break
        
        #y_prev = y_cur    
        
        x1_min_prev = x1_min
        x2_min_prev = x2_min

        j += 1
                     
    return (x1_min, x2_min, j)

<p>Поиск решения посредством стохастического градиентного спуска</p>

In [ ]:
alpha = 0.05
#err = 0.0001
startX1 = 4
startX2 = -2

In [ ]:
stochasticGradientDescent2D_Lite(w1, w2, alpha, f, df_x1_i, df_x2_i, None, startX1, startX2)

<p>Исследование влияния значения коэффициента альфа</p>

<p><b><i>Начальная точка 1</i></b></p>

<img src="img/sgd-two-var-alpha-right.png">

<p><b><i>Начальная точка 2</i></b></p>

<img src="img/sgd-two-var-alpha-left.png">

<p>Время выполнения стохастического градиентного спуска</p>

In [ ]:
alpha = 0.5
startX1 = 4
startX2 = -2

In [ ]:
%timeit stochasticGradientDescent2D_Lite(w1, w2, alpha, f, df_x1_i, df_x2_i, None, startX1, startX2)

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Оптимизация и SciPy</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
from scipy.optimize import minimize, fmin_ncg, fmin, fmin_cg

<p>The downhill simplex algorithm. Nelder-Mead method </p>

In [ ]:
st_point = (4,-2)

res = fmin(f_2d, st_point, retall=True, disp=True)

<p>Nonlinear conjugate gradient algorithm</p>

In [ ]:
st_point = (4,-2)

res = fmin_cg(f_2d, st_point, retall=True, disp=True)

Nelder-Mead

In [ ]:
res = minimize(f_2d, np.array([2,3]), method="nelder-mead", options={"disp": True})
res

Powell

In [ ]:
res = minimize(f_1d, 8, method="powell", options={"xtol": 1e-8, "disp": True})
res

<p>Broyden-Fletcher-Goldfarb-Shanno algorithm</p>

In [ ]:
res = minimize(f, 8, method="BFGS", options={"disp": True})
res

Newton-CG

In [ ]:
res = minimize(f, 8, method="Newton-CG", options={"disp": True})
res

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>SciPy</p>
<a href = "http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.misc.derivative.html">scipy.misc.derivative</a><br>
<a href = "http://www.scipy-lectures.org/advanced/mathematical_optimization/">Mathematical optimization: finding minima of functions</a><br>

<a href = "https://books.google.ru/books?id=gl2kCAAAQBAJ&pg=PA84&lpg=PA84&dq=python+gradient+field&source=bl&ots=-3OAAkvVxC&sig=t_m9sqbKiV4I0iPXh_7GqTOtd2I&hl=ru&sa=X&ved=0ahUKEwjVmPmQ6dTKAhWEiSwKHY_fBEkQ6AEITjAG#v=onepage&q=python%20gradient%20field&f=false">A Student's Guide to Python for Physical Modeling</a><br>
<a href = "https://books.google.ru/books?id=K0B8BwAAQBAJ&pg=PA241&lpg=PA241&dq=python+gradient+field&source=bl&ots=Np0wRoLbEP&sig=TCjpF12bZwjZ6BeeAj6KmE1EHlQ&hl=ru&sa=X&ved=0ahUKEwjVmPmQ6dTKAhWEiSwKHY_fBEkQ6AEIVDAH#v=onepage&q=python%20gradient%20field&f=false">Mathematics and Python Programming</a><br>



<p>Дополнительно</p>
<a href = "http://mit.spbau.ru/files/scipy.pdf">Python. Библиотеки numpy, scipy, matplotlib, PIL</a><br>